https://www.kaggle.com/datasets/luisfredgs/imdb-ptbr?resource=download

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from numpy.lib.function_base import vectorize
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
resume = pd.read_csv("data/imdb-reviews-pt-br.csv")
resume.head()

FileNotFoundError: ignored

In [ ]:

train, test, class_train, class_test = train_test_split(resume.text_pt, 
                                                              resume.sentiment,
                                                              random_state=42)
#regression = LogisticRegression()
#regression.fit(treino, classe_treino)
#acuracia = regression.score(teste, classe_teste)
#print(acuracia)
#Essa parte do código da erro como exemplo.

In [ ]:
classification = resume["sentiment"].replace(["neg","pos"], [0,1])
classification

In [ ]:
resume["classificacao"] = classification
resume.head()

In [ ]:
texto = ["Assisti um filme ótimo", "Assisti um filme ruim"]

vectorize = CountVectorizer(lowercase=False)

bag_of_words = vectorize.fit_transform(texto)

In [ ]:
vectorize.get_feature_names_out()

In [ ]:
vectorize = CountVectorizer(lowercase=False, max_features=50)
bag_of_words = vectorize.fit_transform(resume.text_pt)
print(bag_of_words.shape)

In [ ]:
def classify_text(text, column_text, column_classification):
  vectorize = CountVectorizer(lowercase=False, max_features=50)
  bag_of_words = vectorize.fit_transform(text[column_text])
  train, test, class_train, class_test = train_test_split(bag_of_words, 
                                                              text[column_classification],
                                                              random_state=42)
  
  regression = LogisticRegression()
  regression.fit(train, class_train)
  return regression.score(test, class_test)
   
print(classify_text(resume, "text_pt","classificacao"))


#4 Word Cloud

In [ ]:
%matplotlib inline

from wordcloud import WordCloud

all_words = ' '.join([texto for texto in resume.text_pt])

wordcloud = WordCloud(width=800, height=500,
                      max_font_size=110,
                      collocations=False).generate(all_words)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,7))
plt.imshow(wordcloud, )
plt.axis("off")
plt.show()




In [ ]:
resume.query("sentiment == 'pos'")

In [ ]:
def word_cloud_neg(text, column_text):
  text_neg = text.query("sentiment == 'neg'")
  all_words = ' '.join([texto for texto in text_neg[column_text]])
  wordcloud = WordCloud(width=800, height=500,
                        max_font_size=110,
                        collocations=False).generate(all_words)
                        
  plt.figure(figsize=(10,7))
  plt.imshow(wordcloud, )
  plt.axis("off")
  plt.show()

def word_cloud_pos(text, column_text):
  text_pos = text.query("sentiment == 'pos'")
  all_words = ' '.join([texto for texto in text_pos[column_text]])
  wordcloud = WordCloud(width=800, height=500,
                        max_font_size=110,
                        collocations=False).generate(all_words)
                        
  plt.figure(figsize=(10,7))
  plt.imshow(wordcloud, )
  plt.axis("off")
  plt.show()


In [ ]:
word_cloud_neg(resume, "text_pt")

In [ ]:
word_cloud_pos(resume, "text_pt")

In [ ]:
texto = ["um filme bom","um filme ruim"]
frequency = nltk.FreqDist(texto)
frequency

In [ ]:

texto = "um filme bom um filme ruim"
token_space = tokenize.WhitespaceTokenizer()
token_phrase = token_space.tokenize(texto)
token_phrase

In [ ]:
token_phrase = token_space.tokenize(all_words)
frequency = nltk.FreqDist(token_phrase)
df_freq = pd.DataFrame({"Word": list(frequency.keys()),
                        "Frequency": list(frequency.values())})
nlarge = df_freq.nlargest(columns="Frequency", n=10)

In [ ]:
import seaborn as sns

plt.figure(figsize=(12,8))
ax = sns.barplot(data = nlarge, x = "Word", y = "Frequency", color="gray")
ax.set(ylabel = "Count")
plt.show()


In [ ]:
def pareto_bar(text, cloumn_text,quantity):

  all_words = ' '.join([text for text in text[cloumn_text]])
  token_phrase = token_space.tokenize(all_words)
  frequency = nltk.FreqDist(token_phrase)
  df_freq = pd.DataFrame({"Word": list(frequency.keys()),
                        "Frequency": list(frequency.values())})
  nlarge = df_freq.nlargest(columns="Frequency", n=quantity)
  plt.figure(figsize=(12,8))
  ax = sns.barplot(data = nlarge, x = "Word", y = "Frequency", color="gray")
  ax.set(ylabel = "Count")
  plt.show()


In [ ]:
pareto_bar(resume, "text_pt", 10)

In [ ]:
stop_words = nltk.corpus.stopwords.words("portuguese")
processed_phrase = list()

for opinion in resume.text_pt:
  new_phrase = list()
  text_words = token_space.tokenize(opinion)
  for word in text_words:
    if word not in stop_words:
      new_phrase.append(word)
  processed_phrase.append(' '.join(new_phrase))

resume["tratamento_1"] = processed_phrase

In [ ]:
resume.head()

In [ ]:
classify_text(resume,"tratamento_1","classificacao")

In [ ]:
pareto_bar(resume, "tratamento_1",10)

#5 Tokenização


In [ ]:
from nltk import tokenize
from string import punctuation

pontuacao = list()
for ponto in punctuation:
  pontuacao.append(ponto)

pontuacao_stopwords = pontuacao + stop_words

processed_phrase = list()
for opinion in    resume["tratament_1"]:
  new_phrase = list()
  text_words = token_punctuation.tokenize(opinion)
  for word in text_words:
    if word not in pontuacao_stopwords:
      new_phrase.appen(word)
  frase_processada.append(' '.join(new_phrase))

resume["tratament_2"] = processed_phrase

In [ ]:
import unidecode

sem_acentos = [unidecode.unidecode(text) for text in resume["tratamento_2"]]

In [ ]:
stopwords_sem_acento = [unidecode.unidecode(text) for text in pontuacao_stopwords]

In [ ]:
resume["tratamento_3"] = sem_acentos

processed_phrase = list()
for opinion in resume["tratament_3"]:
  new_phrase = list()
  text_words = token_punctuation.tokenize(opinion)
  for word in text_words:
    if word not in pontuacao_stopwords:
      new_phrase.appen(word)
  frase_processada.append(' '.join(new_phrase))

resume["tratament_3"] = processed_phrase

In [ ]:
accuracy_tratamento3 = classify_text(resume, "tratamento_3", "classificacao")
print(accuracy_tratamento3)

In [ ]:
word_cloud_neg(resume, "tratamento_3")

In [ ]:
word_cloud_pos(resume, "tratamento_3")

In [ ]:
pareto_bar(resume, "tratamento_3")

In [ ]:
processed_phrase = list()
for opinion in resume["tratament_3"]:
  new_phrase = list()
  opinion = opinion.lower()
  text_words = token_punctuation.tokenize(opinion)
  for word in text_words:
    if word not in pontuacao_stopwords:
      new_phrase.appen(word)
  frase_processada.append(' '.join(new_phrase))

resume["tratament_4"] = processed_phrase

In [ ]:
accuracy_tratamento3 = classify_text(resume, "tratamento_4", "classificacao")
print(accuracy_tratamento3)

In [ ]:
word_cloud_neg(resume, "tratamento_4")

In [ ]:
word_cloud_pos(resume, "tratamento_4")

In [ ]:
pareto_bar(resume, "tratamento_4")

#6 Stemmer

In [ ]:
stemmer = nltk.RSLPStemmer()

In [ ]:
processed_phrase = list()
for opinion in resume["tratament_4"]:
  new_phrase = list()
  
  text_words = token_punctuation.tokenize(opinion)
  for word in text_words:
    if word not in pontuacao_stopwords:
      new_phrase.appen(stemmer.stem(word))
  frase_processada.append(' '.join(new_phrase))

resume["tratament_5"] = processed_phrase

In [ ]:
accuracy_tratamento3 = classify_text(resume, "tratamento_5", "classificacao")
print(accuracy_tratamento3)

In [ ]:
word_cloud_neg(resume, "tratamento_5")

In [ ]:
word_cloud_pos(resume, "tratamento_5")

In [ ]:
pareto_bar(resume, "tratamento_5")

#7 TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(lowercase=FAlse, max_features=50)
regression = LogisticRegression()

In [ ]:
raw_tfidf = tfidf.fit_transform(resenha["text_pt"])
train,test, class_train, class_test = train_test_split(raw_tfidf,
                                                      resenha["classificacao"],
                                                      random_state=42)

regression.fit(train, class_train)
accuracy_raw_tfidf = regression.score(test, class_test)
print(accuracy_raw_tfidf)

In [ ]:
tratados_tfidf = tfidf.fit_transform(resenha["tratamento_5"])
train,test, class_train, class_test = train_test_split(tratados_tfidf,
                                                      resenha["classificacao"],
                                                      random_state=42)

regression.fit(train, class_train)
accuracy_tratados_tfidf = regression.score(test, class_test)
print(accuracy_tratados_tfidf)

#8 Ngrams


In [ ]:
from nltk import ngrams

In [ ]:
tfidf  = TfidfVectorizer(lowercase=False, ngram_range=(1,2))
vector_tfidf = tfidf.fit_transform(resenha["tratamento_5"])
train, test, class_train, class_test = train_test_split(vector_tfidf,
                                                        resenha["classificacao"],
                                                        random_state=42)
regression.fit(train,class_train)
accuracy_ngrams = regression.score(test,class_test)
print(accuracy_ngrams)